Colin's version of train_lm.ipynb from https://github.com/krypticmouse/double-bind-training/commit/9f0351b7c162a48175b07ae717cb6680517e488f

In [1]:
!git clone https://github.com/krypticmouse/double-bind-training.git
%cd double-bind-training

Cloning into 'double-bind-training'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 69 (delta 29), reused 48 (delta 11), pack-reused 0
Unpacking objects: 100% (69/69), 729.41 KiB | 3.30 MiB/s, done.
/content/double-bind-training


In [2]:
!git checkout train-lm-adapter

Branch 'train-lm-adapter' set up to track remote branch 'train-lm-adapter' from 'origin'.
Switched to a new branch 'train-lm-adapter'


In [3]:
! pip install adapter-transformers seqeval ptvsd wandb datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 60.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [4]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## download and preprocess training data from MAFAND
https://huggingface.co/datasets/masakhane/mafand is the Mafand dataset. It is intended for machine translation but we can just use it for language modeling. 

['en-amh', 'en-hau', 'en-ibo', 'en-kin', 'en-lug', 'en-nya', 'en-pcm', 'en-sna', 'en-swa', 'en-tsn', 'en-twi', 'en-xho', 'en-yor', 'en-zul', 'fr-bam', 'fr-bbj', 'fr-ewe', 'fr-fon', 'fr-mos', 'fr-wol'] are the available translation sets. 

In [5]:
from datasets import load_dataset

dataset = load_dataset("masakhane/mafand", "en-swa")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset mafand downloaded and prepared to /root/.cache/huggingface/datasets/masakhane___mafand/en-swa/1.0.0/67304918ceaeb8cf2cfa0cc76a684248c282da43b067fa7a1cb69965d4fedbb7. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

### (implemented) Convert translation to language modeling set

Pull out one language, convert to language modeling set. Language modeling sets are monolingual, and each data item has a "text" field like this example from https://colab.research.google.com/github/Adapter-Hub/adapter-transformers/blob/master/notebooks/01_Adapter_Training.ipynb

```
dataset = load_dataset("rotten_tomatoes")
dataset['train'][0]
{'label': 1,
 'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'}
```

We should be able to just pull out all the ones with one language from "train", and all the ones from "validation" and all the ones from "test", then reformat them to be formatted like that. 

In [12]:
flat_dataset = dataset.flatten()
flat_dataset

DatasetDict({
    train: Dataset({
        features: ['translation.en', 'translation.swa'],
        num_rows: 30782
    })
    validation: Dataset({
        features: ['translation.en', 'translation.swa'],
        num_rows: 1791
    })
    test: Dataset({
        features: ['translation.en', 'translation.swa'],
        num_rows: 1835
    })
})

In [13]:
language_code = "swa"

for split in flat_dataset:
  flat_dataset[split] = flat_dataset[split].remove_columns('translation.en')

In [14]:
for split in flat_dataset:
  split_strings = []
  for data_item in flat_dataset[split]:
    values = data_item.values()
    for value in values: 
      # print(value)
      split_strings.append(value)
  with open(f"{split}.txt", "w") as spf:
    spf.writelines('\n'.join(split_strings))

In [15]:
!pwd

/content/double-bind-training


In [16]:
!rm -rf /tmp/test-mlm

In [11]:
!python train_lm_adapter.py \
    --model_name_or_path roberta-base \
    --train_file train.txt \
    --validation_file validation.txt \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --train_adapter \
    --do_train \
    --do_eval \
    --num_train_epochs 1 \
    --report_to "wandb" \
    --run_name "adapter-training-lm-test" \
    --output_dir /tmp/test-mlm \
    --dataset_language "swa"

wandb: Currently logged in as: krypticmouse (double-bind-ner). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.9
wandb: Run data is saved locally in /content/double-bind-training/wandb/run-20230127_171837-pmnc6gmg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run lunar-moon-14
wandb: ⭐️ View project at https://wandb.ai/double-bind-ner/training-lm-test-run
wandb: 🚀 View run at https://wandb.ai/double-bind-ner/training-lm-test-run/runs/pmnc6gmg
INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay

In [17]:
!ls -alh /tmp/test-mlm

ls: cannot access '/tmp/test-mlm': No such file or directory


In [ ]:
%env MAX_LENGTH=164
%env ADAPTER_MODEL=roberta-base
%env OUTPUT_DIR=swa_sample
%env BATCH_SIZE=32
%env NUM_EPOCHS=1
%env SAVE_STEPS=10000
%env SEED=1

!CUDA_VISIBLE_DEVICES=1 python3 train_ner_adapter.py --data_dir data/swa/ \
--model_type roberta \
--model_name_or_path $ADAPTER_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--num_train_epochs $NUM_EPOCHS \
--per_gpu_train_batch_size $BATCH_SIZE \
--save_steps $SAVE_STEPS --learning_rate 5e-4 \
--seed $SEED \
--dataset_language "ner-swa" \
--path_to_adapter /tmp/test-mlm \
--do_train \
--do_eval \
--do_predict

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

MessageError: ignored

In [ ]:
!mkdir -p /gdrive/MyDrive/masakkhane/double-bind/dancing-fish-5/

In [ ]:
!cp -rv /tmp/test-mlm/* /gdrive/MyDrive/masakkhane/double-bind/dancing-fish-5/

'/tmp/test-mlm/all_results.json' -> '/gdrive/MyDrive/masakkhane/double-bind/dancing-fish-5/all_results.json'
'/tmp/test-mlm/checkpoint-1000' -> '/gdrive/MyDrive/masakkhane/double-bind/dancing-fish-5/checkpoint-1000'
'/tmp/test-mlm/checkpoint-1000/mlm' -> '/gdrive/MyDrive/masakkhane/double-bind/dancing-fish-5/checkpoint-1000/mlm'
'/tmp/test-mlm/checkpoint-1000/mlm/adapter_config.json' -> '/gdrive/MyDrive/masakkhane/double-bind/dancing-fish-5/checkpoint-1000/mlm/adapter_config.json'
'/tmp/test-mlm/checkpoint-1000/mlm/pytorch_adapter.bin' -> '/gdrive/MyDrive/masakkhane/double-bind/dancing-fish-5/checkpoint-1000/mlm/pytorch_adapter.bin'
'/tmp/test-mlm/checkpoint-1000/mlm/head_config.json' -> '/gdrive/MyDrive/masakkhane/double-bind/dancing-fish-5/checkpoint-1000/mlm/head_config.json'
'/tmp/test-mlm/checkpoint-1000/mlm/pytorch_model_head.bin' -> '/gdrive/MyDrive/masakkhane/double-bind/dancing-fish-5/checkpoint-1000/mlm/pytorch_model_head.bin'
'/tmp/test-mlm/checkpoint-1000/tokenizer_config.j